## 事件流处理器

In [ ]:
from illufly.io import EventBlock
EventBlock("Chunk", "hi")

### 实现流交换

基于大模型的AI应用中经常要求模型流式返回，我们有一种实现流输出的标准实现。

In [ ]:
from illufly.types import Runnable

class MyRun(Runnable):
    def call(*args, **kwargs):
        yield "hi\n"
        yield "illufly!\n"

r = MyRun()
r()

In [ ]:
from illufly.types import Runnable

class MyRun(Runnable):
    def call(*args, **kwargs):
        yield ["hi\n", "illufly!\n"]

r = MyRun()
r()

### Runnable 事件流

In [ ]:
from illufly.types import Runnable, EventBlock

class MyRun(Runnable):
    def call(*args, **kwargs):
        yield EventBlock("chunk", "hi, ")
        yield EventBlock("chunk", "illufly")
        yield EventBlock("chunk", "!")

def handle(block, **kwargs):
    print(block.json)

r = MyRun(handlers=[handle])
r("hi")

### ChatAgent 事件流

In [ ]:
from illufly.types import Runnable, EventBlock
from illufly.chat import FakeLLM
from illufly.io import log, alog

def handle(block, **kwargs):
    print("[", block.block_type, "]", block.text)

r = FakeLLM(handlers=[handle], sleep=0.5)
r("hi")

In [ ]:
from illufly.types import Runnable, EventBlock
from illufly.chat import FakeLLM
from illufly.io import log, alog

r = FakeLLM(handlers=[alog], sleep=0.1)
await r("hi")

### 推送事件流

In [ ]:
from illufly.chat import FakeLLM

llm = FakeLLM(name="if1", handlers=[])
llm.create_new_thread()

async for b in llm("给我写一篇200字小作文", generator="async", sleep=0.1):
    print(b)

## 嵌套 BaseAgent

In [ ]:
from illufly.types import BaseAgent, PromptTemplate
from illufly.chat import ChatQwen
from illufly.flow import ReAct

def get_city(location: str):
    """由任意地名或地址描述查询出所在的城市"""
    return "广州"

def get_weather(city: str):
    """我可以查询城市的天气情况。city必须是城市名称。"""
    return f'{city}天气不错，一直是大晴天'

def booking(request: str):
    """我可以根据你的需求预订球场"""
    return '我已经帮你预订好了，祝你玩得愉快'

store = {}
flow = ReAct(
    ChatQwen(name="react-planner", tools=[get_city, get_weather, booking])
)

for b in flow("我在鸿蒙公司，帮我查询一下天气情况", verbose=False, handlers=[], generator="sync"):
    print(b)

## Events History

In [1]:
from illufly.core.events_history import BaseEventsHistory

In [2]:
h = BaseEventsHistory()

In [3]:
h.store

{'20241116-69681-1108-0000': {'threads': set(), 'callings': {}}}

In [4]:
h.last_events_history_id

'20241116-69681-1108-0000'